In [8]:
import numpy as np
import pandas as pd
from scipy.stats import norm

In [ ]:
pip install scipy

In [10]:
T=6
N=1000
drugs = ['M','ST','SG','AGI', 'PA']
muhat0 = [0.32, 0.28, 0.30, 0.26, 0.21]
sigmahat0 = [0.12, 0.09, 0.17, 0.15, 0.11]

In [14]:
# initial belief
s0 = {x: (muhat0[k], 1/sigmahat0[k]**2) 
      for k, x in enumerate(drugs)}

In [16]:
s0

{'M': (0.32, 69.44444444444444),
 'ST': (0.28, 123.4567901234568),
 'SG': (0.3, 34.60207612456747),
 'AGI': (0.26, 44.44444444444444),
 'PA': (0.21, 82.64462809917356)}

In [18]:
sigma_test = 5
phi_test = 1/sigma_test **2
true_mu_low = 0.15
true_mu_high = 0.45

In [22]:
truth_seed = pd.read_excel('Lecture 6 Continuous Uniform 0-1.xlsx', sheet_name = 'Diabetes_truth')
observed_seed = pd.read_excel('Lecture 6 Continuous Uniform 0-1.xlsx', sheet_name = 'Diabetes_observed')

In [24]:
truth_seed.head(3)

,U1,U2,U3,U4,U5
0,0.784088,0.169264,0.814389,0.441520,0.634123
1,0.430665,0.193144,0.177082,0.209839,0.769090
2,0.244497,0.264560,0.457426,0.960882,0.539002


In [26]:
i = 0 # patient 0
true_mu = {x: (true_mu_high - true_mu_low) * truth_seed.iloc[i,k] + true_mu_low
           for k,x in enumerate(drugs)}
true_mu

{'M': 0.3852264401201282,
 'ST': 0.2007790989944614,
 'SG': 0.39431679991274926,
 'AGI': 0.28245613357632243,
 'PA': 0.3402370172507302}

In [29]:
curr_state = s0.copy()
curr_state

{'M': (0.32, 69.44444444444444),
 'ST': (0.28, 123.4567901234568),
 'SG': (0.3, 34.60207612456747),
 'AGI': (0.26, 44.44444444444444),
 'PA': (0.21, 82.64462809917356)}

In [33]:
theta = 1 # to be optimized
t = 0
# interval estimation
ie_stats = {x: curr_state[x][0] + theta / np.sqrt(curr_state[x][1])
    for x in drugs}
ie_stats

{'M': 0.44,
 'ST': 0.37,
 'SG': 0.47,
 'AGI': 0.41000000000000003,
 'PA': 0.31999999999999995}

In [37]:
opt_drug = max(ie_stats, key = ie_stats.get)
opt_drug

'SG'

In [39]:
# sample
w = norm.ppf(observed_seed.iloc[i,t],
            true_mu[opt_drug],
            sigma_test)
w

8.346003889297709

In [47]:
# update belief
phihat = curr_state[opt_drug][1]+ phi_test
muhat = (curr_state[opt_drug][1]*curr_state[opt_drug][0]+phi_test * w) /phihat
curr_state[opt_drug] = (muhat,phihat)
curr_state


{'M': (0.32, 69.44444444444444),
 'ST': (0.28, 123.4567901234568),
 'SG': (0.3092904407465251, 34.64207612456747),
 'AGI': (0.26, 44.44444444444444),
 'PA': (0.21, 82.64462809917356)}

In [55]:
def cal_obj(theta):
    obj = np.zeros(N)
    for i in range(N):
        true_mu = {x: (true_mu_high - true_mu_low) * truth_seed.iloc[i,k] + true_mu_low
                   for k,x in enumerate(drugs)}
        curr_state = s0.copy()
        for t in range(T):
            ie_stats = {x: curr_state[x][0] + theta / np.sqrt(curr_state[x][1])
                        for x in drugs}
        opt_drug = max(ie_stats, key = ie_stats.get)
        w = norm.ppf(observed_seed.iloc[i,t],
                    true_mu[opt_drug],
                    sigma_test)
        phihat = curr_state[opt_drug][1]+ phi_test
        muhat = (curr_state[opt_drug][1]*curr_state[opt_drug][0]+phi_test * w) /phihat
        curr_state[opt_drug] = (muhat,phihat)
        obj[i] +=w
    return np.mean(obj)

In [65]:
theta_grid = np.arange(0, 2.1, 0.2)
a1c = []
from tqdm import tqdm
for theta in tqdm(theta_grid):
    a1c.append(cal_obj(theta))
opt_val = max(a1c)
opt_theta = theta_grid[a1c.index(opt_val)]

100%|██████████| 11/11 [00:12<00:00,  1.11s/it]


In [67]:
opt_val

0.19942929165740844

In [69]:
opt_drug

'SG'